#### 1. Form a dataframe using the first group of product names: 

In [8]:
from difflib import SequenceMatcher
import difflib
from fuzzywuzzy import fuzz
import pandas as pd
from numpy import triu, ones

t1 = "Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (Up To 95Mb/S Read)Memory Card"
t2 = "Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb Sdsdxpa 064G"
t3 = "Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk Sdsdxxg 064G X46)95 90Mb/S Read Write Speed Video Speed C10 U3 V30"

tg1 = "SANDISK SDC CLASS 10 EXTREME PRO 95MB/S SERIES 64GB"
tg2 = "SANDISK SDC CLASS 10 EXTREME PRO UHS-II 275MB/S SERIES 64GB"

In [5]:
s1 = pd.Series([t1,t2,t3], index=range(3))

In [6]:
dfs = pd.DataFrame([s1.values.tolist()] * s1.shape[0], index=s1.index, columns=s1.index)
dfs = dfs.applymap(lambda x: list([x]))

In [9]:
dfd = dfs + dfs.T
dfd = dfd.mask(triu(ones(dfd.shape)).astype(bool))
dfd

,0,1,2
0,NaN,NaN,NaN
1,[Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (U...,NaN,NaN
2,[Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (U...,[Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb S...,NaN


#### 2. Compute the similarity between product name in the first group using fuzzywuzzy: 

In [15]:
def dist(ss):
    if type(ss) == list:
        return fuzz.partial_ratio(ss[0], ss[1])  

dfg = dfd.applymap(dist)
dfg

,0,1,2
0,NaN,NaN,None
1,54.0,NaN,None
2,56.0,70.0,None


#### 3. Group similar object (which are assumed to be the same product)

In [43]:
grouplist = []
for i, row in dfd[dfg > 50].iterrows():
    for r in row.values.tolist():
        if type(r) == list:
            grouplist.append(r)

In [44]:
def merge(lsts):
    sets = [set(lst) for lst in lsts if lst]
    merged = 1
    while merged:
        merged = 0
        results = []
        while sets:
            common, rest = sets[0], sets[1:]
            sets = []
            for x in rest:
                if x.isdisjoint(common):
                    sets.append(x)
                else:
                    merged = 1
                    common |= x
            results.append(common)
        sets = results
    return sets

In [45]:
merge(grouplist)

[{'Sandisk Extreme Pro 64Gb Sdxc Uhs I U3 V30 (Up To 95Mb/S Read)Memory Card',
  'Sandisk Extreme Pro Sd Uhs I 64Gb Card (Sdsk Sdsdxxg 064G X46)95 90Mb/S Read Write Speed Video Speed C10 U3 V30',
  'Sandisk Sd Extreme Pro Uhs U1 Class 10 64Gb Sdsdxpa 064G'}]

#### 4. Form the dataframe with similar above to compute the similarity between all the product names in the first and second group 

#### 5.  compute the similarity between all the product names in the first and second group

#### 6. Find the strongest similarity between the elements in the grouplist and the elements in the second group and match the product names 